# Image Classification of Dogs vs. Cats With PyTorch

### Imports & Environment

In [27]:
import os
import random
import collections
import shutil
import time
import glob
import csv
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image

from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/Shared drives/DANI IVAN EDGAR MARÇAL/DEEPLEARNING/Edgar - Dani/Deep_Learning2020/final/try2

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data_5_animals'



#%cd  valid

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/Shared drives/DANI IVAN EDGAR MARÇAL/DEEPLEARNING/Edgar - Dani/Deep_Learning2020/final/try2


### Config & Hyperparameters

In [0]:
# paths
data_path = DATA_HOME_DIR + '/' 
split_train_path = data_path + '/train/'
full_train_path = data_path + '/train_full/'
valid_path = data_path + '/validation/'
test_path = DATA_HOME_DIR + '/test/test/'
saved_model_path = ROOT_DIR + '/models/'
submission_path = ROOT_DIR + '/submissions/'


# data


# model
nb_runs = 1
nb_aug = 3

lr = 1e-4
clip = 0.001
#can change this
archs = ["resnet101"]
#archs = ["vgg11"]
#pretrainedModel=False
pretrainedModel=True
epochs = 10
batch_size = 128
#--
model_names = sorted(name for name in models.__dict__ if name.islower() and not name.startswith("__"))
best_prec1 = 0

In [29]:
model_names

['_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'utils',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn',
 'video',
 'wide_resnet101_2',
 'wide_resnet50_2']

### Helper Functions for Training

In [0]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()
    end = time.time()
    
    # switch to train mode
    model.train()
    
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
  
        print("inside train "+str(i))
        target = target.cuda()
        image_var = torch.autograd.Variable(images)
        label_var = torch.autograd.Variable(target)

        # compute y_pred
        y_pred = model(image_var)
        loss = criterion(y_pred, label_var)
       # loss.requires_grad = True

        # measure accuracy and record loss
        prec1, temp_var = accuracy(y_pred, target, topk=(1, 1))
        losses.update(loss, images.size(0))
        acc.update(prec1, images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

In [0]:
def validate(val_loader, model, criterion, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (images, labels) in enumerate(val_loader):
        labels = labels.cuda()
        
        print("inside validate "+str(i))
        image_var = torch.autograd.Variable(images)
        label_var = torch.autograd.Variable(labels)

        # compute y_pred
        y_pred = model(image_var)
        loss = criterion(y_pred, label_var)
        
        #loss.requires_grad = True

        # measure accuracy and record loss
        prec1, temp_var = accuracy(y_pred, labels, topk=(1, 1))
        losses.update(loss, images.size(0))
        acc.update(prec1, images.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    print('   * EPOCH {epoch} | Accuracy: {acc.avg:.3f} | Loss: {losses.avg:.3f}'.format(epoch=epoch,
                                                                                         acc=acc,
                                                                                         losses=losses))

    return acc.avg

In [0]:
def test(test_loader, model):
    csv_map = collections.defaultdict(float)
    csv_type = []
    # switch to evaluate mode
    model.eval()
    
    for aug in range(nb_aug):
        print("   * Predicting on test augmentation {}".format(aug + 1))
        
        for i, (images, filepath) in enumerate(test_loader):
            # pop extension, treat as id to map
            filepath = os.path.splitext(os.path.basename(filepath[0]))[0]
            
            print("inside test "+filepath)
            #filepath = int(filepath)

            print("inside test "+str(i))


            image_var = torch.autograd.Variable(images)
            y_pred = model(image_var)
            # get the index of the max log-probability
            smax = nn.Softmax()
            smax_out = smax(y_pred)[0]
            ardilla_prob = smax_out.data[0]
            caballo_prob = smax_out.data[1]
            elefante_prob = smax_out.data[2]
            mariposa_prob = smax_out.data[3]
            oveja_prob = smax_out.data[4]
            print("this image " + filepath + " have: ")

            string1=np.around(ardilla_prob.cpu(), decimals=4)
            string1 = np.clip(string1, clip, 1-clip)
            print("ardilla " + str(string1.data.cpu().numpy().round(decimals=4)) + "")

            string2=np.around(caballo_prob.cpu(), decimals=4)
            string2 = np.clip(string2, clip, 1-clip)
            print("caballo " + str(string2.data.cpu().numpy().round(decimals=4))  + "")

            string3=np.around(elefante_prob.cpu(), decimals=4)
            string3 = np.clip(string3, clip, 1-clip)
            print("elefante " + str(string3.data.cpu().numpy().round(decimals=4))  + "")

            string4=np.around(mariposa_prob.cpu(), decimals=4)
            string4 = np.clip(string4, clip, 1-clip)
            print("mariposa " + str(string4.data.cpu().numpy().round(decimals=4)) + "")

            string5=np.around(oveja_prob.cpu(), decimals=4)
            string5 = np.clip(string5, clip, 1-clip)
            print("oveja " + str(string5.data.cpu().numpy().round(decimals=4))  + "")

            ##############################################################
            #prob = dog_prob
            #if cat_prob > dog_prob:
            #    prob = 1 - cat_prob
            #prob = np.around(prob.cpu(), decimals=4)
            #prob = np.clip(prob, clip, 1-clip)
            # mirar como añadir mas columnas csv_map[filepath] += ("ardilla")
            lista = [ardilla_prob, caballo_prob, elefante_prob, mariposa_prob, oveja_prob]
            index = lista.index(max(lista))
            maximo = max(lista)
            if index == 0:
                csv_map[filepath] += (string1.data.cpu().numpy() / nb_aug)
                if maximo > 0.55:
                  csv_type.append("es una ardilla")
                else:
                  csv_type.append("puede ser ardilla")
            elif index == 1:
                csv_map[filepath] += (string2.data.cpu().numpy() / nb_aug)
                if maximo > 0.55:
                  csv_type.append("es un caballo")
                else:
                  csv_type.append("puede ser caballo")

            elif index == 2:
                csv_map[filepath] += (string3.data.cpu().numpy() / nb_aug)
                if maximo > 0.55:
                  csv_type.append("es un elefante")
                else:
                  csv_type.append("puede ser elefante")

            elif index == 3:
                csv_map[filepath] += (string4.data.cpu().numpy() / nb_aug)
                if maximo > 0.55:
                  csv_type.append("es una mariposa")
                else:
                  csv_type.append("puede ser mariposa")

            elif index == 4:
                csv_map[filepath] += (string5.data.cpu().numpy() / nb_aug)
                if maximo > 0.55:
                  csv_type.append("es una oveja")
                else:
                  csv_type.append("puede ser oveja")
            
            csv_map[filepath] = csv_map[filepath].round(decimals=4)
            #csv_map[filepath] += (string1.data.cpu().numpy() / nb_aug).round(decimals=4)
            #csv_type.append("ardilla")
            ##############################################################

    sub_fn = submission_path + '60_{0}epoch_{1}clip_{2}runs'.format(epochs, clip, nb_runs)
    
    for arch in archs:
        sub_fn += "_{}".format(arch)
        
    print("Writing Predictions to CSV...")
    with open(sub_fn + '.csv', 'w') as csvfile:
        fieldnames = ['id', 'porcentaje', 'tipo']
        csv_w = csv.writer(csvfile, delimiter=',')
        csv_w.writerow(('id', 'porcentaje', 'tipo'))
        for index, row in enumerate(sorted(csv_map.items())):
            csv_w.writerow([row, csv_type[index]])
    print("Done.")

In [0]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best_dani.pth.tar')      

In [0]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [0]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    global lr
    lr = lr * (0.1**(epoch // 30))
    for param_group in optimizer.state_dict()['param_groups']:
        param_group['lr'] = lr


def accuracy(y_pred, y_actual, topk=(1, )):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = y_actual.size(0)

    _, pred = y_pred.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(y_actual.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res

In [0]:
class TestImageFolder(data.Dataset):
    def __init__(self, root, transform=None):
        images = []
        for filename in sorted(glob.glob(test_path + "*.jpeg")):
            images.append('{}'.format(filename))

        self.root = root
        self.imgs = images
        self.transform = transform

    def __getitem__(self, index):
        filename = self.imgs[index]
        img = Image.open(os.path.join(self.root, filename))
        if self.transform is not None:
            img = self.transform(img)
        return img, filename

    def __len__(self):
        return len(self.imgs)

In [0]:
def shear(img):
    width, height = img.size
    m = random.uniform(-0.05, 0.05)
    xshift = abs(m) * width
    new_width = width + int(round(xshift))
    img = img.transform((new_width, height), Image.AFFINE,
                        (1, m, -xshift if m > 0 else 0, 0, 1, 0),
                        Image.BICUBIC)
    return img

### Main Training Loop

In [0]:
def main(mode="train", resume=False):
    
    global best_prec1
    
    for arch in archs:

        # create model
        print("=> Starting {0} on '{1}' model".format(mode, arch))
        model = models.__dict__[arch](pretrained=pretrainedModel)
        # Don't update non-classifier learned features in the pretrained networks
        for param in model.parameters():
            param.requires_grad = False
        # Replace the last fully-connected layer
        # Parameters of newly constructed modules have requires_grad=True by default
        # Final dense layer needs to replaced with the previous out chans, and number of classes
        # in this case -- resnet 101 - it's 2048 with two classes (cats and dogs)
        
        if arch.startswith('resnet'):
          model.fc = nn.Linear(2048, 5)
        if arch.startswith('vgg11'):
          model.classifier[6]=nn.Linear(4096, 5)

        if arch.startswith('alexnet') or arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()
            
        # optionally resume from a checkpoint
        if resume:
            if os.path.isfile(resume):
                print("=> Loading checkpoint '{}'".format(resume))
                checkpoint = torch.load(resume)
                start_epoch = checkpoint['epoch']
                best_prec1 = checkpoint['best_prec1']
                model.load_state_dict(checkpoint['state_dict'])
                print("=> Loaded checkpoint (epoch {})".format(checkpoint['epoch']))
            #else:
                #print("=> No checkpoint found at '{}'".format(args.resume))

        cudnn.benchmark = True

        # Data loading code
        traindir = split_train_path
        valdir = valid_path
        testdir = test_path

        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        train_loader = data.DataLoader(
            datasets.ImageFolder(traindir,
                                 transforms.Compose([
                                     # transforms.Lambda(shear),
                                     transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     normalize,
                                 ])),
            batch_size=batch_size,
            shuffle=True,
            num_workers=4,
            pin_memory=False)

        val_loader = data.DataLoader(
            datasets.ImageFolder(valdir,
                                 transforms.Compose([
                                     transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     normalize,
                                 ])),
            batch_size=batch_size,
            shuffle=True,
            num_workers=4,
            pin_memory=False)

        test_loader = data.DataLoader(
            TestImageFolder(testdir,
                            transforms.Compose([
                                # transforms.Lambda(shear),
                                transforms.Resize(256),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                normalize,
                            ])),
            batch_size=1,
            shuffle=False,
            num_workers=1,
            pin_memory=False)
        
        
        if mode == "test":
            test(test_loader, model)
            return
        
        # define loss function (criterion) and pptimizer
        criterion = nn.CrossEntropyLoss().cuda()
        
        if mode == "validate":
            validate(val_loader, model, criterion, 0)
            return

        print(model)
      
        if arch.startswith('resnet'):
          optimizer = optim.Adam(model.module.fc.parameters(), lr, weight_decay=1e-4)
        if arch.startswith('vgg11'):
          optimizer = optim.Adam(model.parameters(), lr, weight_decay=1e-4)
       
       

        for epoch in range(epochs):
            adjust_learning_rate(optimizer, epoch)

            # train for one epoch
            train(train_loader, model, criterion, optimizer, epoch)

            # evaluate on validation set
            prec1 = validate(val_loader, model, criterion, epoch)

            # remember best Accuracy and save checkpoint
            is_best = prec1 > best_prec1
            best_prec1 = max(prec1, best_prec1)
            save_checkpoint({
                'epoch': epoch + 1,
                'arch': arch,
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
            }, is_best)

### Run Train

In [39]:
main(mode="train")

=> Starting train on 'vgg11' model
VGG(
  (features): DataParallel(
    (module): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): ReLU(inplace=True)
      (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (14): ReLU(inplace

KeyboardInterrupt: ignored

In [0]:
main(mode="validate", resume='model_best_dani.pth.tar')

### Run Test

In [0]:
main(mode="test", resume='model_best_dani.pth.tar')